In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-imda/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

68448

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
with open(files[0]) as fopen:
    d = json.load(fopen)
    
d[0]

{'predict_ms': [50258,
  50282,
  50360,
  50365,
  430,
  1538,
  29216,
  7408,
  11,
  43365,
  3680,
  13877,
  992,
  3779,
  16281,
  717,
  289,
  657,
  282,
  12711,
  1706,
  66,
  12584,
  1706,
  73,
  9286,
  17289,
  74,
  892,
  449,
  7691,
  1988,
  23171,
  1026,
  40463,
  545,
  23059,
  569,
  5581,
  717,
  302,
  4579,
  3077,
  13,
  50715,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'predict_en': [50258,
  50259,
  50360,
  50365,
  4928,
  4104,
  82,
  365,
  3779,
  6352,
  645,
  14658,
  281,
  853,
  5145,
  641,
  1281,
  12126,
  2507,
  613,
  21688,
  4358,
  13,
  50715,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'score_ms': 10.375,
 'score_en': 8.75,
 'filename': 'IMDA-STT/part1-mp3/000010101.mp3'}

In [10]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_en'] > minimum_score:
                a = np.array(data[i]['predict_en'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() >= 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|en|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [11]:
results = loop((files[:10], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 10/10 [01:23<00:00,  8.36s/it]


In [12]:
len(results)

362

In [13]:
results[-1]['new_text']

'<|startoftranscript|><|en|><|transcribe|><|0.00|> I heard you loud and clear when you said it the first time.<|3.84|><|endoftext|>'

In [14]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
tokenizer.tokenize(results[-1]['new_text'])

['<|startoftranscript|>',
 '<|en|>',
 '<|transcribe|>',
 '<|0.00|>',
 'ĠI',
 'Ġheard',
 'Ġyou',
 'Ġloud',
 'Ġand',
 'Ġclear',
 'Ġwhen',
 'Ġyou',
 'Ġsaid',
 'Ġit',
 'Ġthe',
 'Ġfirst',
 'Ġtime',
 '.',
 '<|3.84|>',
 '<|endoftext|>']

In [16]:
tokenizer.decode(tokenizer(results[-1]['new_text'])['input_ids'], decode_with_timestamps = True)

'<|startoftranscript|><|notimestamps|><|startoftranscript|><|en|><|transcribe|><|0.00|> I heard you loud and clear when you said it the first time.<|3.84|><|endoftext|><|endoftext|>'

In [17]:
results = mp.multiprocessing(files, loop, cores = 100)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [18]:
len(results)

1861125

In [19]:
results[0]

{'new_text': '<|startoftranscript|><|en|><|transcribe|><|0.00|> Households with target sets were encouraged to try keeping their water consumption below these designated levels.<|7.00|><|endoftext|>',
 'audio_filename': 'IMDA-STT/part1-mp3/000010101.mp3'}

In [20]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [21]:
with open('prepared-imda.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 1861125/1861125 [00:06<00:00, 267450.32it/s]


In [22]:
!ls -lh prepared-imda.jsonl

-rw-r--r-- 1 ubuntu ubuntu 520M Apr 26 08:48 prepared-imda.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj='prepared-imda.jsonl',
    path_in_repo='prepared-imda.jsonl',
    repo_id='mesolitica/pseudolabel-imda-large-v3-timestamp',
    repo_type='dataset',
)